In [ ]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602926 sha256=fc9dd81e6fc642416fa01e02fb56db3cb18c0bac18cf87b6b7959110023b7ae0
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [ ]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=E6Fvu5yG4EaNaCCrirFYc74F2b4f2Q&prompt=consent&token_usage=remote&access_type=offline&code_challenge=CTAzADaBL_4XpWmEy9eoOGYDE0t0sFnk0eA7rtd3aqo&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQlEd8yqsPKdOkmYHqcHkc_M9suXZ20jNemjaU8aABIh1iKSp5wpNmMHcevi62TqWaAzWw

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [ ]:
client = bigquery.Client(project='basedosdados')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
df = pd.read_csv("siorg.csv",  decimal=',')
df

,sigla_cargo_funcao,nome_cargo_funcao,nivel_cargo_funcao,remuneracao,cce_unitario
0,CCE,Cargo Comissionado Executivo,CCE 1 18,17327.65,6.41
1,CCE,Cargo Comissionado Executivo,CCE 1 17,16944.9,6.27
2,CCE,Cargo Comissionado Executivo,CCE 1 16,15688.92,5.81
3,CCE,Cargo Comissionado Executivo,CCE 1 15,13623.39,5.04
4,CCE,Cargo Comissionado Executivo,CCE 1 14,11652.88,4.31
...,...,...,...,...,...
253,DAS,Grupo-Direção e Assessoramento Superiores,DAS 103 2,3440.75,1.27
254,DAS,Grupo-Direção e Assessoramento Superiores,DAS 103 1,2701.46,1.0
255,MEST,Ministro de Estado,MEST,30934.7,11.45
256,PR,Presidente da República,PR,30934.7,11.45


In [ ]:
df['remuneracao'] = pd.to_numeric(df['remuneracao'], errors='coerce')
df['cce_unitario'] = pd.to_numeric(df['cce_unitario'], errors='coerce')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sigla_cargo_funcao  258 non-null    object 
 1   nome_cargo_funcao   258 non-null    object 
 2   nivel_cargo_funcao  258 non-null    object 
 3   remuneracao         258 non-null    float64
 4   cce_unitario        258 non-null    float64
dtypes: float64(2), object(3)
memory usage: 10.2+ KB


In [ ]:
schema = [
    bigquery.SchemaField('sigla_cargo_funcao', 'STRING', description='Sigla do cargo ou função.'),
    bigquery.SchemaField('nome_cargo_funcao', 'STRING', description='Nome do cargo ou função.'),
    bigquery.SchemaField('nivel_cargo_funcao', 'STRING', description='Nível do cargo ou função.'),
    bigquery.SchemaField('remuneracao', 'FLOAT', description='Remuneração associada ao cargo ou função.'),
    bigquery.SchemaField('cce_unitario', 'FLOAT', description='CCE unitário associado ao cargo ou função.')
]

In [ ]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('estrutura_organizacional_carreiras')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
table_ref = dataset_ref.table('SIORG_remuneracao_direcao_assessoramento') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=ecbc3972-934e-4482-b14a-c09a6b3cae73>